In [1]:
import os
import glob
import time
import datetime
import argparse
import numpy as np
import pandas as pd
from collections import OrderedDict

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

# pytorch package
import torch
import torch.nn as nn
from torchvision.io import read_image
from torch.utils.data import DataLoader

# predefined class
from Models import mobilenet, ResNet18, ResNet34, ResNet50
from Training import SolarFlSets, HSS2, TSS, F1Pos, HSS_multiclass, TSS_multiclass, train_loop, test_loop_cp, oversample_func
from Cp import compute_cov_length, conformity_score

In [2]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
torch.backends.cudnn.benchmark = True
print('1st check cuda..')
print('Number of available device', torch.cuda.device_count())
print('Current Device:', torch.cuda.current_device())
print('Device:', device)

1st check cuda..
Number of available device 2
Current Device: 0
Device: cuda


In [3]:
model = mobilenet().to(device)

# 1. First, let's check the keys in your state dict
file = f'/workspace/Project/Multi_imagery_SFPred/Results/CV/regmodel/Mobilenet_Het_freezeFalse_202411_train12_cp3_test4_lr2.4_decayval4.0.pth'

# becuase of the parallel processing we have different prix for the weights
saved_state_dict = torch.load(file)['model_state_dict']
new_state_dict = OrderedDict()
for k, v in saved_state_dict.items():
    name = k.replace("module.", "") # Remove the 'model.' prefix
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)

model.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/root/.cache/pypoetry/virtualenvs/fulldisk-pj-izM7TedW-py3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/fulldisk-pj-izM7TedW-py3.8/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


mobilenet(
  (model): MobileNetV3(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          )
        )
      )
      (2): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
 

In [4]:
basic_dir = '/workspace/data/hetero_data/'
euv = 'euv/compressed/304/2013/10/25/EUV-304.2013.10.25_00.00.00.jpg'
conti = 'hmi/compressed/continuum/2013/10/25/HMI-CTnuum.2013.10.25_00.00.00.jpg'
mag = 'hmi/compressed/mag/2013/10/25/HMI-Mag.2013.10.25_00.00.00.jpg'

image1 = read_image(basic_dir + euv).float()
image2 = read_image(basic_dir + conti).float()
image3 = read_image(basic_dir + mag).float()
total_img = [image1, image2, image3]
image = torch.stack(total_img).squeeze()
image = image / 255

In [9]:
with torch.no_grad():
    output = model(image.unsqueeze(0).to(device))

In [11]:
probabilities = torch.nn.functional.softmax(output[0], dim=0)

In [12]:
probabilities

tensor([0.0451, 0.1516, 0.4433, 0.3600], device='cuda:0')